The Liquidity-Adjusted Value at Risk (LAVaR) is calculated as:

LAVaR=VaR+LC

VaR (Value at Risk) measures the maximum potential loss over a given time frame with a certain confidence level.
LC (Liquidity Cost) estimates the cost associated with liquidating the asset position over the desired time frame, taking into account the market impact, bid-ask spread, and depth of the market for the asset.

Calculation of LC (Liquidity Cost):
LC =(Bid-Ask Spread Cost+Market Impact Cost)×Position Size

Bid-Ask Spread Cost can be determined from current market data.
Market Impact Cost can be estimated based on historical data regarding how previous large transactions have affected the asset's price.

The LAVaR metric provides a more holistic view of risk by incorporating the liquidity aspect. This allows the fund to understand not just the risk of a position, but how easily and at what cost the position can be unwound. With insights from LAVaR, the fund can make more informed decisions about asset allocation, especially in terms of diversifying into less liquid assets or sectors. By adjusting for liquidity, LAVaR allows for a more accurate assessment of the fund's performance, particularly in stress scenarios where liquidity can significantly affect returns. LAVaR can help in meeting regulatory requirements related to liquidity risk management, ensuring the fund maintains a healthy liquidity profile. Demonstrating a sophisticated approach to measuring and managing risk, including liquidity risk, can increase investor confidence in the fund's management capabilities.

The Liquidity-Adjusted Value at Risk (LAVaR) offers a comprehensive risk metric that integrates the cost of liquidity into the traditional VaR model. For student-managed investment funds, this metric is particularly valuable for balancing risk and liquidity considerations, thereby supporting more informed and strategic investment decisions.

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf

#Start by getting stock data using yfinance
def fetch_stock_data(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data_low = stock_data['Low']
    stock_data_high = stock_data['High']
    stock_data_volume = stock_data['Volume']
    
    if stock_data.empty or 'Adj Close' not in stock_data.columns:
        print(f"Error fetching data for {ticker}. Please check the provided stock ticker and date range.")
        return None, None, None, None

    #Drops missing values
    returns = stock_data['Adj Close'].pct_change().dropna()

    if returns.empty:
        print(f"No returns data available for {ticker} within the specified date range.")
        return None, None, None, None
    
    return stock_data_low, stock_data_high, stock_data_volume, returns

def calculate_bid_ask_spread_cost(stock_data_low, stock_data_high, position_size):
    #Extract bid and ask prices from the provided stock data
    bid_price = stock_data_low
    ask_price = stock_data_high
    
    #Calculate bid-ask spread
    bid_ask_spread = ask_price - bid_price
    
    #Calculate bid-ask spread cost based on the position size
    bid_ask_spread_cost = bid_ask_spread * position_size
    
    #Return the calculated cost
    return bid_ask_spread_cost

def calculate_market_impact_cost(stock_data_volume, position_size):
    #Extract trading volume data from the provided stock data
    trading_volume = stock_data_volume
    
    #Calculate average daily trading volume
    average_daily_volume = np.mean(trading_volume)
    
    #Calculate the impact ratio based on the position size and average daily volume
    impact_ratio = position_size / average_daily_volume
    
    #Calculate market impact cost using the impact ratio and trading volume
    market_impact_cost = impact_ratio * trading_volume
    
    #Return the calculated cost
    return market_impact_cost

def calculate_liquidity_cost(ticker, start_date, end_date, stock_data_low, stock_data_high, stock_data_volume, position_size):
    #Calculate Bid-Ask Spread Cost
    bid_ask_spread_cost = calculate_bid_ask_spread_cost(stock_data_low, stock_data_high, position_size)
    
    #Calculate Market Impact Cost
    market_impact_cost = calculate_market_impact_cost(stock_data_volume, position_size)
    
    #Calculate Liquidity Cost
    liquidity_cost = bid_ask_spread_cost + market_impact_cost
    return liquidity_cost

#Calculating VaR at 95% confidence level
def calculate_var(returns, confidence_level=0.95):
    sorted_returns = np.sort(returns)
    var_index = int(np.ceil((1 - confidence_level) * len(sorted_returns)))
    var = -sorted_returns[var_index]
    return var

def calculate_lavar(var, liquidity_cost):
    #VAR + LC
    lavar = var + liquidity_cost
    return lavar

def main():
    ticker = input("Enter the stock ticker symbol: ").upper()
    start_date = input("Enter the start date (YYYY-MM-DD): ")
    end_date = input("Enter the end date (YYYY-MM-DD): ")
    position_size = float(input("Enter the position size: "))

    stock_data_low, stock_data_high, stock_data_volume, stock_returns = fetch_stock_data(ticker, start_date, end_date)

    print("Stock Returns:")
    print(stock_returns)

    #Calculate VaR
    var = calculate_var(stock_returns)

    calculate_bid_ask_spread_cost(stock_data_low, stock_data_high, position_size)
    calculate_market_impact_cost(stock_data_volume, position_size)
    calculate_liquidity_cost(ticker, start_date, end_date, stock_data_low, stock_data_high, stock_data_volume, position_size)

    #Calculate Liquidity Cost
    liquidity_cost_value = calculate_liquidity_cost(ticker, start_date, end_date, stock_data_low, stock_data_high, stock_data_volume, position_size)

    #Calculate LAVaR Ratio
    lavar_ratio = calculate_lavar(var, liquidity_cost_value)
    print(f"LAVaR Ratio for {ticker}: {lavar_ratio}")


if __name__ == "__main__":
    main()